In [1]:
# install Pytorch: https://pytorch.org/

# import necessary modules
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# download MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor(),
                                          download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST\raw\train-images-idx3-ubyte.gz to ../../data/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST\raw\train-labels-idx1-ubyte.gz to ../../data/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data/MNIST\raw



In [2]:
# divide the dataset into batches
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [3]:
# Convolutional neural network (two convolutional layers)
class UDA_CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(UDA_CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(
                    in_channels = 1,
                    out_channels = 16,
                    kernel_size=5,
                    stride=1,
                    padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
      
model = UDA_CNN()

In [4]:
# Loss function
criterion = nn.CrossEntropyLoss()
# optimizer to update network parameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
total_step = len(train_loader)
for epoch in range(2):
    for i, (images, labels) in enumerate(train_loader):
        images = images
        labels = labels
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 5, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1741
Epoch [1/5], Step [200/600], Loss: 0.1104
Epoch [1/5], Step [300/600], Loss: 0.0447
Epoch [1/5], Step [400/600], Loss: 0.0705
Epoch [1/5], Step [500/600], Loss: 0.0269
Epoch [1/5], Step [600/600], Loss: 0.0234
Epoch [2/5], Step [100/600], Loss: 0.1064
Epoch [2/5], Step [200/600], Loss: 0.0239
Epoch [2/5], Step [300/600], Loss: 0.0604
Epoch [2/5], Step [400/600], Loss: 0.0367
Epoch [2/5], Step [500/600], Loss: 0.0808
Epoch [2/5], Step [600/600], Loss: 0.0422


In [5]:
# Test the model
model.eval()  # turn off batchnorm layer
with torch.no_grad(): # turn off gradient computation
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
    
# Save the model checkpoint
torch.save(model.state_dict(), 'model.uda')

Test Accuracy of the model on the 10000 test images: 98.81 %
